In [1]:
from Multiclass_SVM import MultiSVM,np
from pyspark import SparkContext,SparkConf
from sklearn.base import copy
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def fit_in(x,svm,classes):
    for it in x:
        X=it[:-1].reshape(1,it.shape[0]-1)
        y=np.array([it[-1]])
        y=y.astype('int')
        svm.partial_fit(X,y,classes)
    yield svm
def update_in(m1,m2):
    new_mod=copy.deepcopy(m1)
    new_mod.coefs+=m2.coefs
    new_mod.intercept+=m2.intercept
    return new_mod

def avg_coefs_in(svm,numpart):
    svm.coefs/=numpart
    svm.intercept/=numpart
    return svm

In [4]:
conf=SparkConf().setAppName("MultiSVM-SGD").setMaster('local[4]')
sc=SparkContext(conf=conf)

In [11]:
X=mnist.train.images[:10000,:]
y=mnist.train.labels[:10000]
X=np.hstack((X,y.reshape((y.shape[0],1))))

In [17]:
max_iter=250
svm=MultiSVM(l=0.0001)
n_classes=np.unique(y).shape[0]
prev_coef=np.zeros((n_classes,X.shape[1]-1))
prev_intercept=np.zeros(n_classes)
tol=0.0001

In [18]:
for i in range(max_iter):
    dat=sc.parallelize(X)
    svm=dat.mapPartitions(lambda x:fit_in(x,svm,n_classes)).reduce(lambda m1,m2:update_in(m1,m2))
    svm=avg_coefs_in(svm,dat.getNumPartitions())
    diff_coef=svm.coefs-prev_coef
    diff_intercept=svm.intercept-prev_intercept
    coef_norm=np.linalg.norm(diff_coef,ord='fro')**2
    gradient_sum=np.sqrt(coef_norm+np.sum(diff_intercept**2))
    if gradient_sum<tol:
        break
    prev_coef=svm.coefs
    prev_intercept=svm.intercept
    np.random.shuffle(X)
print(svm.coefs)
print(svm.intercept)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[-0.4975 -0.5775  0.3175 -0.6875  0.275   2.3375 -0.6475  1.485  -1.5575
 -0.4475]


In [5]:
X1=5*np.random.random((400,2))-3
y1=0*np.ones(X1.shape[0])
X2=6*(np.random.random((400,2)))+7
y2=np.ones(X2.shape[0])
X3=25*(np.random.random((400,2)))+20
y3=2*np.ones(X2.shape[0])
y=np.hstack((y1,y2,y3))
X=np.vstack((X1,X2,X3))
y=y.reshape(y.shape[0],1)
X=np.hstack((X,y))

In [19]:
svm.predict(X[:,:-1])

In [20]:
svm.support_values

array([[ -0.41626969,   2.41882535,   2.98104061, ...,  -4.7069067 ,
          8.38391854,   2.32072387],
       [ -6.33236063,   7.06645323,   3.32200786, ...,  -0.76602734,
          2.42033267,  -0.75587957],
       [  2.8272625 , -13.79380218,  16.5408056 , ...,   0.32003107,
         -0.39485462,  -2.73944811],
       ...,
       [ -3.35433523,  -4.15712646,  -1.3338482 , ...,  12.50814927,
          2.52833762,   7.6241167 ],
       [ -4.5692111 ,  -8.63654805,   5.08905853, ...,   0.03290686,
         10.58070074,   3.50060918],
       [ -2.06975151,   0.88825057,  -1.21010485, ...,  -2.24074222,
          7.59434841,   1.60494859]])

In [21]:
np.mean(svm.predictions==X[:,-1])

0.9966